In [152]:
#import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pyarrow as pa
import seaborn as sns
from datetime import timedelta, datetime

dir = Path(r"S:\Fackler_OSS_364376\data\IRB-364376-v1-230215")

In [136]:
fp = dir.joinpath('EHR', 'd_flo_measures.csv.gz')
dict = pd.read_csv(fp, compression="gzip")
# dict.head()
# dict['disp_name'].unique

names = ["State Behavioral Scale",
"-3 Unresponsive", 
"-2 Responsive to noxious stimuli", 
"-1 Responsive to gentle touch or voice",
"0 Awake and Able to calm",
"+1 Restless and difficult to calm",
"+2 Agitated",
"State Behavioral Scale (SBS)"]

dict[dict['disp_name'].isin(names)]

,flo_meas_id,flo_meas_name,site_row_id,record_state_c,allow_comp_yn,disp_name,abbr_p,row_typ_c,chg_trg_type_c,val_type_c,...,flo_row_status_c,row_type,val_type,cross_enc_yn,question_id,question_prompt,question_required_yn,context_c,store_calc_data_yn,show_abbr_c
64810,304080015,G JHM IP PEDS STATE BEHAVIORAL SCALE,NaN,NaN,NaN,State Behavioral Scale,NaN,2.0,NaN,NaN,...,0.0,Flowsheet Group,NaN,N,NaN,NaN,NaN,1.0,NaN,NaN
64811,304080016,R JHM IP AD PEDS SB UNRESPONSIVE,NaN,NaN,NaN,-3 Unresponsive,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64812,304080017,R JHM IP AD PEDS SB RESPONSIVE TO NOXIOUS STIMULI,NaN,NaN,NaN,-2 Responsive to noxious stimuli,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64813,304080018,R JHM IP AD PEDS SB RESPONSIVE TO GENTLE TOUCH...,NaN,NaN,NaN,-1 Responsive to gentle touch or voice,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64814,304080019,R JHM IP AD PEDS SB AWAKE AND ABLE TO CALM,NaN,NaN,NaN,0 Awake and Able to calm,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64815,304080020,R JHM IP AD PEDS SB RESTLESS AND DIFFICULT TO ...,NaN,NaN,NaN,+1 Restless and difficult to calm,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64816,304080021,R JHM IP AD PEDS SB AGITATED,NaN,NaN,NaN,+2 Agitated,NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
64817,304080022,G JHM IP AD PEDS STATE BEHAVIORAL SCALE (SBS),NaN,NaN,NaN,State Behavioral Scale (SBS),NaN,2.0,NaN,NaN,...,0.0,Flowsheet Group,NaN,N,NaN,NaN,NaN,1.0,NaN,NaN
64818,304080023,R JHM IP AD PEDS STATE BEHAVIORAL SCALE (SBS),NaN,NaN,NaN,State Behavioral Scale (SBS),NaN,1.0,NaN,8.0,...,0.0,Data,Custom List,N,NaN,NaN,NaN,1.0,NaN,NaN
66624,1570400708,G BH STATE BEHAVIORAL SCALE,NaN,NaN,NaN,State Behavioral Scale (SBS),NaN,2.0,NaN,NaN,...,0.0,Flowsheet Group,NaN,N,NaN,NaN,NaN,1.0,NaN,NaN


In [137]:
# note: flowsheet record flow_meas_id as meas_id
# note: SBS score values are only stored in these fields

fmid = [304080016, 304080017, 304080018, 304080019, 304080020, 304080021]

In [138]:
# connect with feather file
fp = dir.joinpath('EHR', 'ptsd_record.csv.gz')

ptsd_record = pd.read_csv(fp, compression="gzip")
# print(ptsd_record.dtypes)
# ptsd_record.head(2)
patients_ptsd = ptsd_record['pat_enc_csn_sid'].unique()

# load flow table of all patient EHR records
fp = dir.joinpath('EHR', 'flowsheet.csv.gz')

data = pd.read_csv(fp, compression="gzip")
data = data.drop(columns = ['meas_comment', 'meas_template_id'])
# Note: pandas took 50 seconds to load the table. Consider porting to PySpark RDD

In [139]:
# print(data.dtypes)
sbs = data[data['meas_id'].isin(fmid)]
print(sbs.shape)
# 25878 entries

# calculate sbs score from offset
sbs['SBS'] = sbs['meas_id'] - 304080019
sbs = sbs.drop(columns=['meas_value', 'meas_id'])
sbs['recorded_time'] = pd.to_datetime(sbs['recorded_time'], format='%Y-%m-%d %H:%M:%S')
sbs_indiv = sbs.groupby('pat_enc_csn_sid')

# Identify patients with SBS and waveform data
patients_ehr = list(sbs_indiv.groups.keys())

patients = list(set(patients_ehr) & set(patients_ptsd))
print(len(patients))
# 581 patients with both EHR and waveform

(25878, 5)
581


c:\Program Files\miniconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [140]:
p = patients[0]

files = ptsd_record[ptsd_record['pat_enc_csn_sid'] == p]
files['start_time'] = pd.to_datetime(files['start_time'], format='%Y-%m-%d %H:%M:%S')
files['end_time'] = pd.to_datetime(files['end_time'], format='%Y-%m-%d %H:%M:%S')
files.sort_values('start_time')

devices = files['device']
filename = files['filename'] + '.feather'
startime = files['start_time']
endtime = files['end_time']

dfs = []

size = 0

for (d, fn, t0, t1) in zip(devices, filename, startime, endtime):
    fp_device = 'vitals-' + d.lower()
    fp_p = str(p)[-3:] # last 3 digit of pat_enc_csn_sid is the subfolder
    fp = dir.joinpath('ptsd-phi', fp_device, fp_p, fn)
    
    # add datetime overlap handling
    df = pd.read_feather(fp, columns=None, use_threads=True, storage_options=None)
    df['delta'] = pd.to_timedelta(df['dts'], unit='s')
    df['recorded_time'] = df['delta'] + startime.iloc[0]
    size += df.shape[0]

    
    # df presents time as delta seconds
    dfs.append(df)
patient_multi = pd.concat(dfs, axis=0)

# add SBS
sbs_p = sbs_indiv.get_group(p).sort_values('recorded_time')
sbs_p = sbs_p.drop(columns=['osler_sid', 'pat_enc_csn_sid'])
# sbs_p

c:\Program Files\miniconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Program Files\miniconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,dts,AR1-D,AR1-M,AR1-R,AR1-S,CVP2,HR,NBP-D,NBP-M,NBP-R,NBP-S,PVC,RR,SPO2-%,SPO2-R,delta,recorded_time,SBS
0,58740.0,42.0,54.0,108.0,72.0,8.0,108.0,NaN,NaN,NaN,NaN,NaN,0.0,99.0,108.0,0 days 16:19:00,2017-07-08 08:37:00,NaN
1,58800.0,44.0,56.0,118.0,74.0,10.0,130.0,NaN,NaN,NaN,NaN,NaN,9.0,99.0,110.0,0 days 16:20:00,2017-07-08 08:38:00,NaN
2,58860.0,50.0,64.0,132.0,80.0,12.0,122.0,NaN,NaN,NaN,NaN,NaN,16.0,100.0,124.0,0 days 16:21:00,2017-07-08 08:39:00,NaN
3,58920.0,44.0,58.0,108.0,76.0,2.0,104.0,NaN,NaN,NaN,NaN,0.0,0.0,100.0,106.0,0 days 16:22:00,2017-07-08 08:40:00,NaN
4,58980.0,44.0,58.0,100.0,76.0,4.0,98.0,NaN,NaN,NaN,NaN,NaN,0.0,99.0,98.0,0 days 16:23:00,2017-07-08 08:41:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,37380.0,NaN,NaN,NaN,NaN,NaN,114.0,NaN,NaN,NaN,NaN,NaN,37.0,92.0,114.0,0 days 10:23:00,2017-07-08 02:41:00,NaN
3831,54960.0,NaN,NaN,NaN,NaN,NaN,118.0,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,0 days 15:16:00,2017-07-08 07:34:00,NaN
3832,55020.0,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,NaN,NaN,0.0,32.0,NaN,NaN,0 days 15:17:00,2017-07-08 07:35:00,NaN
3833,55080.0,NaN,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,0.0,30.0,NaN,NaN,0 days 15:18:00,2017-07-08 07:36:00,NaN


### Approach 1
Combine SBS and waveforms into a single pandas dataframe

In [ ]:
patient_merged = pd.merge(patient_multi, sbs_p, how='left')
# patient_merged

# plot some data from the table
sns.lineplot(x='dts', y='SBS', data=patient_merged, markers=True)
sns.lineplot(x = 'dts', y = 'HR', data=patient_merged)

### Approach 2
Segment x seconds of data before each SBS event, and label with SBS score

We will use this ML training

In [145]:
sbs_p

,recorded_time,SBS
25700849,2017-07-07 16:30:00,0
25756688,2017-07-07 20:00:00,-1
11170452,2017-07-08 00:00:00,-1
25705412,2017-07-08 04:00:00,-1
25700480,2017-07-08 12:00:00,0
25723824,2017-07-08 16:00:00,0


In [164]:
for i in range(len(sbs_p)):
    t1 = sbs_p['recorded_time'].iloc[i]
    t0 = t1 - timedelta(seconds=30*60) # 30 minutes prior

    x = patient_multi.loc[(patient_multi['recorded_time'] >= t0)
                      & (patient_multi['recorded_time'] <= t1)]
    y = sbs_p.iloc[i]

    print(x.head())
    print(y)

     dts  AR1-D  AR1-M  AR1-R  AR1-S  CVP2     HR  NBP-D  NBP-M  NBP-R  NBP-S  \
0    0.0    NaN    NaN    NaN    NaN   NaN  114.0    NaN    NaN    NaN    NaN   
1   60.0    NaN    NaN    NaN    NaN   NaN  112.0    NaN    NaN    NaN    NaN   
2  120.0    NaN    NaN    NaN    NaN   NaN  114.0    NaN    NaN    NaN    NaN   
3  180.0    NaN    NaN    NaN    NaN   NaN  112.0    NaN    NaN    NaN    NaN   
4  240.0    NaN    NaN    NaN    NaN   NaN  112.0    NaN    NaN    NaN    NaN   

   PVC    RR  SPO2-%  SPO2-R           delta       recorded_time  
0  0.0  31.0    97.0   114.0 0 days 00:00:00 2017-07-07 16:18:00  
1  0.0  28.0    98.0   114.0 0 days 00:01:00 2017-07-07 16:19:00  
2  0.0  23.0    98.0   114.0 0 days 00:02:00 2017-07-07 16:20:00  
3  0.0  29.0    98.0   112.0 0 days 00:03:00 2017-07-07 16:21:00  
4  0.0  28.0    98.0   112.0 0 days 00:04:00 2017-07-07 16:22:00  
recorded_time    2017-07-07 16:30:00
SBS                                0
Name: 25700849, dtype: object
       